In [8]:
import sys
!{sys.executable} -m pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.8 MB/s eta 0:00:00a 0:00:01


In [9]:
from psycopg2 import connect as connect
from datetime import datetime
import numpy as np

In [31]:
try:
    conn = connect(
        dbname="dvdrental", user="postgres", password="postgres", host="0.0.0.0", port=5432
    )
    print("\nСоединение с БД установлено", conn, "\n")
except Exception as e:
    print("БД не доступна. Проверьте Docker", e)


Соединение с БД установлено <connection object at 0x1075142e0; dsn: 'user=postgres password=xxx dbname=dvdrental host=0.0.0.0 port=5432', closed: 0> 



In [11]:
# определение функций для выполнения запросов к БД
# получение данных
def query_get(conn, query: str) -> dict:
    with conn.cursor() as curs:
        curs.execute(query)
        result = curs.fetchall()
        return result

# передача данных
def query_set(conn, query: str):
    with conn.cursor() as curs:
        curs.execute(query)
        conn.commit()
        print(f"выполнен запрос к БД: '{query}' \n")

# получение данных и их вывод на экран
def query_n_print(conn, query_read: str):
    print("\n", f"Выполнен запрос к БД - '{query_read}'", ":\n", "-" * 50)
    result = query_get(conn, query_read)
    for row in result:
        print(row)

In [12]:
query_n_print(conn, """select * from category limit 20""")
query_n_print(conn, """select * from film_category limit 20""")
query_n_print(conn, """select * from customer where first_name = 'John'""")


 Выполнен запрос к БД - 'select * from category limit 20' :
 --------------------------------------------------
(1, 'Action', datetime.datetime(2006, 2, 15, 9, 46, 27))
(2, 'Animation', datetime.datetime(2006, 2, 15, 9, 46, 27))
(3, 'Children', datetime.datetime(2006, 2, 15, 9, 46, 27))
(4, 'Classics', datetime.datetime(2006, 2, 15, 9, 46, 27))
(5, 'Comedy', datetime.datetime(2006, 2, 15, 9, 46, 27))
(6, 'Documentary', datetime.datetime(2006, 2, 15, 9, 46, 27))
(7, 'Drama', datetime.datetime(2006, 2, 15, 9, 46, 27))
(8, 'Family', datetime.datetime(2006, 2, 15, 9, 46, 27))
(9, 'Foreign', datetime.datetime(2006, 2, 15, 9, 46, 27))
(10, 'Games', datetime.datetime(2006, 2, 15, 9, 46, 27))
(11, 'Horror', datetime.datetime(2006, 2, 15, 9, 46, 27))
(12, 'Music', datetime.datetime(2006, 2, 15, 9, 46, 27))
(13, 'New', datetime.datetime(2006, 2, 15, 9, 46, 27))
(14, 'Sci-Fi', datetime.datetime(2006, 2, 15, 9, 46, 27))
(15, 'Sports', datetime.datetime(2006, 2, 15, 9, 46, 27))
(16, 'Travel', date

In [13]:
# Объединение двух таблиц по полю "category_id", с выводом только тех строк, которые совпадают в обеих таблицах
query = """SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
INNER JOIN film_category f ON c.category_id = f.category_id
ORDER BY id_film
LIMIT 50 OFFSET 50"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
INNER JOIN film_category f ON c.category_id = f.category_id
ORDER BY id_film
LIMIT 50 OFFSET 50' :
 --------------------------------------------------
(12, 'Music', 51, 12)
(9, 'Foreign', 52, 9)
(8, 'Family', 53, 8)
(12, 'Music', 54, 12)
(14, 'Sci-Fi', 55, 14)
(1, 'Action', 56, 1)
(16, 'Travel', 57, 16)
(6, 'Documentary', 58, 6)
(3, 'Children', 59, 3)
(4, 'Classics', 60, 4)
(7, 'Drama', 61, 7)
(6, 'Documentary', 62, 6)
(8, 'Family', 63, 8)
(7, 'Drama', 64, 7)
(11, 'Horror', 65, 11)
(3, 'Children', 66, 3)
(1, 'Action', 67, 1)
(3, 'Children', 68, 3)
(14, 'Sci-Fi', 69, 14)
(2, 'Animation', 70, 2)
(8, 'Family', 71, 8)
(6, 'Documentary', 72, 6)
(14, 'Sci-Fi', 73, 14)
(12, 'Music', 74, 12)
(16, 'Travel', 75, 16)
(12, 'Music', 76, 12)
(13, 'New', 77, 13)
(2, 'Animation', 78, 2)
(7, 'Drama', 79, 7)
(8, 'Family', 80, 8)
(14, 'Sci-Fi', 81, 1

In [14]:
# Объединение двух таблиц по полю "category_id", с выводом всех строк из таблицы category (первой в формуле, т.е. LEFT),
# и тех строк из таблицы film_category, в которых есть совпадения по полю "category_id"
query = """SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
LEFT JOIN film_category f ON c.category_id = f.category_id
ORDER BY id_cat_cat
LIMIT 20"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
LEFT JOIN film_category f ON c.category_id = f.category_id
ORDER BY id_cat_cat
LIMIT 20' :
 --------------------------------------------------
(1, 'Action', 19, 1)
(1, 'Action', 21, 1)
(1, 'Action', 29, 1)
(1, 'Action', 38, 1)
(1, 'Action', 56, 1)
(1, 'Action', 67, 1)
(1, 'Action', 97, 1)
(1, 'Action', 105, 1)
(1, 'Action', 111, 1)
(1, 'Action', 115, 1)
(1, 'Action', 126, 1)
(1, 'Action', 130, 1)
(1, 'Action', 162, 1)
(1, 'Action', 194, 1)
(1, 'Action', 205, 1)
(1, 'Action', 210, 1)
(1, 'Action', 212, 1)
(1, 'Action', 229, 1)
(1, 'Action', 250, 1)
(1, 'Action', 252, 1)


In [15]:
# Объединение двух таблиц по полю "category_id", с выводом всех строк из таблицы category (первой в формуле, т.е. LEFT),
# которых нет в таблице film_category, т.е. не имеющих совпадений по полю "category_id"
query = """SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
LEFT JOIN film_category f ON c.category_id = f.category_id
WHERE f.film_id is NULL
"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
LEFT JOIN film_category f ON c.category_id = f.category_id
WHERE f.film_id is NULL
' :
 --------------------------------------------------


In [25]:
# Полное объединение двух таблиц по полю "category_id", с выводом всех строк обеих таблиц. При отсутствии совпадений
# поля заполняются значениями NULL
# в формуле необязательно использование ключевого слова OUTER
query = """SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
FULL OUTER JOIN film_category f ON c.category_id = f.category_id
LIMIT 50
"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
FULL OUTER JOIN film_category f ON c.category_id = f.category_id
LIMIT 50
' :
 --------------------------------------------------
(6, 'Documentary', 1, 6)
(11, 'Horror', 2, 11)
(6, 'Documentary', 3, 6)
(11, 'Horror', 4, 11)
(8, 'Family', 5, 8)
(9, 'Foreign', 6, 9)
(5, 'Comedy', 7, 5)
(11, 'Horror', 8, 11)
(11, 'Horror', 9, 11)
(15, 'Sports', 10, 15)
(9, 'Foreign', 11, 9)
(12, 'Music', 12, 12)
(11, 'Horror', 13, 11)
(4, 'Classics', 14, 4)
(9, 'Foreign', 15, 9)
(9, 'Foreign', 16, 9)
(12, 'Music', 17, 12)
(2, 'Animation', 18, 2)
(1, 'Action', 19, 1)
(12, 'Music', 20, 12)
(1, 'Action', 21, 1)
(13, 'New', 22, 13)
(2, 'Animation', 23, 2)
(11, 'Horror', 24, 11)
(13, 'New', 25, 13)
(14, 'Sci-Fi', 26, 14)
(15, 'Sports', 27, 15)
(5, 'Comedy', 28, 5)
(1, 'Action', 29, 1)
(11, 'Horror', 30, 11)
(8, 'Family', 31, 8)
(13, 'New', 32, 13)
(7, 'Dra

In [17]:
# Вывод уникальных строк двух таблиц, т.е. только тех строк двух таблиц,
# которые не имеют совпадений по полю "category_id"
# в формуле не используется ключевое слова OUTER
query = """SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
FULL JOIN film_category f ON c.category_id = f.category_id
WHERE c.category_id is NULL OR f.category_id is NULL
"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    c.category_id id_cat_cat,
    c.name name_cat,
    f.film_id id_film,
    f.category_id id_cat_film
FROM
    category c
FULL JOIN film_category f ON c.category_id = f.category_id
WHERE c.category_id is NULL OR f.category_id is NULL
' :
 --------------------------------------------------


In [26]:
# Объединение ТРЕХ таблиц по полю "film_id" (поле есть во всех таблицах)
# с выводом только тех строк, которые совпадают во всех таблицах
query = """SELECT
    i.film_id id_film_inventory,
    i.store_id id_store_inventory,
    f.title title_film,
    fc.category_id id_cat_film
FROM
    film f
INNER JOIN film_category fc ON f.film_id = fc.film_id
INNER JOIN inventory i ON fc.film_id = i.film_id
LIMIT 50"""
query_n_print(conn, query)


 Выполнен запрос к БД - 'SELECT
    i.film_id id_film_inventory,
    i.store_id id_store_inventory,
    f.title title_film,
    fc.category_id id_cat_film
FROM
    film f
INNER JOIN film_category fc ON f.film_id = fc.film_id
INNER JOIN inventory i ON fc.film_id = i.film_id
LIMIT 50' :
 --------------------------------------------------
(1, 1, 'Academy Dinosaur', 6)
(1, 1, 'Academy Dinosaur', 6)
(1, 1, 'Academy Dinosaur', 6)
(1, 1, 'Academy Dinosaur', 6)
(1, 2, 'Academy Dinosaur', 6)
(1, 2, 'Academy Dinosaur', 6)
(1, 2, 'Academy Dinosaur', 6)
(1, 2, 'Academy Dinosaur', 6)
(2, 2, 'Ace Goldfinger', 11)
(2, 2, 'Ace Goldfinger', 11)
(2, 2, 'Ace Goldfinger', 11)
(3, 2, 'Adaptation Holes', 6)
(3, 2, 'Adaptation Holes', 6)
(3, 2, 'Adaptation Holes', 6)
(3, 2, 'Adaptation Holes', 6)
(4, 1, 'Affair Prejudice', 11)
(4, 1, 'Affair Prejudice', 11)
(4, 1, 'Affair Prejudice', 11)
(4, 1, 'Affair Prejudice', 11)
(4, 2, 'Affair Prejudice', 11)
(4, 2, 'Affair Prejudice', 11)
(4, 2, 'Affair Prejudice', 1

In [19]:
# пакетная проверка времени запросов, перемежаемых иными
query_set(conn, """CREATE INDEX IF NOT EXISTS index_length ON film (length)""")
time_with_index_array = np.array([])

for i in range(100):
    for j in range(100):
        start_time = datetime.now()
        query_get(conn, f"""SELECT * FROM film WHERE length = {80 + j}""")
        stop_time = datetime.now()
        time_with_index_array = np.append(
            time_with_index_array,
            [stop_time - start_time],
        )
        query_get(conn, f"""SELECT * FROM film LIMIT {1 + j}""")

query_set(conn, """DROP INDEX IF EXISTS index_length""")
time_witout_index_array = np.array([])

for i in range(100):
    for j in range(100):
        start_time = datetime.now()
        query_get(conn, f"""SELECT * FROM film WHERE length = {80 + j}""")
        stop_time = datetime.now()
        time_witout_index_array = np.append(
            time_witout_index_array,
            [stop_time - start_time],
        )
        query_get(conn, f"""SELECT * FROM film LIMIT {1 + j}""")

print(
    f"\nСреднее время {len(time_witout_index_array)} запросов без индекса = {time_witout_index_array.mean()}",
    f"\nСреднее время {len(time_with_index_array)} запросов c индексом = {time_with_index_array.mean()}",
)

выполнен запрос к БД: 'CREATE INDEX IF NOT EXISTS index_length ON film (length)' 

выполнен запрос к БД: 'DROP INDEX IF EXISTS index_length' 


Среднее время 10000 запросов без индекса = 0:00:00.000202 
Среднее время 10000 запросов c индексом = 0:00:00.000156


In [27]:
# Создание представления в виде таблицы БД
query = """
CREATE OR REPLACE VIEW viewname003 AS
SELECT title, release_year, rental_rate, length, rating
FROM film
WHERE length >= 180"""
query_set(conn, query)

# Вывод представления в виде таблицы БД
query = """
SELECT * 
FROM viewname003
"""
query_n_print(conn, query)

# Вывод первых 10 строу таблицы-представления, отсортированной по полю rental_rate
query = """
SELECT * 
FROM viewname003
ORDER BY rental_rate DESC LIMIT 10"""
query_n_print(conn, query)

выполнен запрос к БД: '
CREATE OR REPLACE VIEW viewname003 AS
SELECT title, release_year, rental_rate, length, rating
FROM film
WHERE length >= 180' 


 Выполнен запрос к БД - '
SELECT * 
FROM viewname003
' :
 --------------------------------------------------
('Alley Evolution', 2006, Decimal('2.99'), 180, 'NC-17')
('Analyze Hoosiers', 2006, Decimal('2.99'), 181, 'R')
('Baked Cleopatra', 2006, Decimal('2.99'), 182, 'G')
('Catch Amistad', 2006, Decimal('0.99'), 183, 'G')
('Chicago North', 2006, Decimal('4.99'), 185, 'PG-13')
('Confidential Interview', 2006, Decimal('4.99'), 180, 'NC-17')
('Conspiracy Spirit', 2006, Decimal('2.99'), 184, 'PG-13')
('Control Anthem', 2006, Decimal('4.99'), 185, 'G')
('Crystal Breaking', 2006, Decimal('2.99'), 184, 'NC-17')
('Darn Forrester', 2006, Decimal('4.99'), 185, 'G')
('Frontier Cabin', 2006, Decimal('4.99'), 183, 'PG-13')
('Gangs Pride', 2006, Decimal('2.99'), 185, 'PG-13')
('Haunting Pianist', 2006, Decimal('0.99'), 181, 'R')
('Home Pity', 2006, D

In [28]:
# Общая продолжительность фильмов, старше 2007 года по алфавиту, первые 10 строк
query = """
SELECT * 
FROM viewname003
WHERE release_year < 2007
ORDER BY title DESC LIMIT 10"""
query_n_print(conn, query)


 Выполнен запрос к БД - '
SELECT * 
FROM viewname003
WHERE release_year < 2007
ORDER BY title DESC LIMIT 10' :
 --------------------------------------------------
('Young Language', 2006, Decimal('0.99'), 183, 'G')
('Worst Banger', 2006, Decimal('2.99'), 185, 'PG')
('Wild Apollo', 2006, Decimal('0.99'), 181, 'R')
('Wife Turn', 2006, Decimal('4.99'), 183, 'NC-17')
('Theory Mermaid', 2006, Decimal('0.99'), 184, 'PG-13')
('Sweet Brotherhood', 2006, Decimal('2.99'), 185, 'R')
('Star Operation', 2006, Decimal('2.99'), 181, 'PG')
('Sorority Queen', 2006, Decimal('0.99'), 184, 'NC-17')
('Sons Interview', 2006, Decimal('2.99'), 184, 'NC-17')
('Something Duck', 2006, Decimal('4.99'), 180, 'NC-17')


In [29]:
#  Ддя получения общей суммы платы за прокат для фильмов длиннее 180 минут,
# сначала формируется новое представление с новой колонкой суммы, 
# и затем по нему осуществляется группировка по рейтингу и сортировка
query = """
CREATE OR REPLACE VIEW viewname005 AS
SELECT rating, sum(rental_rate) OVER (ORDER BY rating) as sum_rental_rate_year
FROM viewname003
ORDER BY rating DESC
"""
query_set(conn, query)
query = """
SELECT rating, sum_rental_rate_year
FROM viewname005
GROUP BY rating, sum_rental_rate_year
ORDER BY rating DESC
"""
query_n_print(conn, query)

выполнен запрос к БД: '
CREATE OR REPLACE VIEW viewname005 AS
SELECT rating, sum(rental_rate) OVER (ORDER BY rating) as sum_rental_rate_year
FROM viewname003
ORDER BY rating DESC
' 


 Выполнен запрос к БД - '
SELECT rating, sum_rental_rate_year
FROM viewname005
GROUP BY rating, sum_rental_rate_year
ORDER BY rating DESC
' :
 --------------------------------------------------
('NC-17', Decimal('151.54'))
('R', Decimal('112.65'))
('PG-13', Decimal('87.74'))
('PG', Decimal('47.86'))
('G', Decimal('31.90'))


In [32]:
query = """
SELECT name
FROM category
WHERE category_id =
(SELECT category_id  FROM film_category WHERE film_id = 
(SELECT film_id FROM film WHERE length = 
(SELECT MIN(length) FROM film)
LIMIT 1)
)
"""
query_n_print(conn, query)


 Выполнен запрос к БД - '
SELECT name
FROM category
WHERE category_id =
(SELECT category_id  FROM film_category WHERE film_id = 
(SELECT film_id FROM film WHERE length = 
(SELECT MIN(length) FROM film)
LIMIT 1)
)
' :
 --------------------------------------------------
('Foreign',)


In [23]:
# закрытие подключения к БД
conn.close()
print("-- программа завершена --")

-- программа завершена --
